## Rainfall with Legend

In [28]:
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.colors as mcolors

# Load the CSV file containing the rainfall data
rainfall_data = pd.read_csv('data/annual_rainfall.csv')

# Load the SVG map of India
with open('svg files/india_rainfall.svg', 'r', encoding='utf-8') as svg_file:
    svg_content = svg_file.read()

# Parse the SVG file with BeautifulSoup
soup = BeautifulSoup(svg_content, 'xml')

# Define a color map (Blues colormap in this case)
cmap = mcolors.LinearSegmentedColormap.from_list("Blues", ["#f7fbff", "#08306b"])

# Normalize the rainfall values to map to colors
min_rainfall = rainfall_data['Rainfall'].min()
max_rainfall = rainfall_data['Rainfall'].max()
norm = mcolors.Normalize(vmin=min_rainfall, vmax=max_rainfall)

# Function to get the color based on rainfall amount
def get_color(rainfall):
    rgba = cmap(norm(rainfall))
    return mcolors.to_hex(rgba)

# Loop over each path in the SVG and check the 'title' attribute
for path in soup.find_all('path'):
    # Check if there is a 'title' attribute inside the path that contains the state name
    if 'title' in path.attrs:
        state_name = path['title'].strip()  # Get the state name from the 'title' attribute
        
        # Look for the state in the rainfall data
        matching_row = rainfall_data[rainfall_data['State'] == state_name]
        
        if not matching_row.empty:
            rainfall = matching_row.iloc[0]['Rainfall']
            color = get_color(rainfall)
            
            # Set the fill color based on rainfall
            path['fill'] = color
            
            # Set a border for the states
            path['stroke'] = "#000000"  # Black border
            path['stroke-width'] = "1"  # 1px border

# Add a legend to the SVG

# Create a new <g> (group) tag to contain the legend elements
legend_group = soup.new_tag('g')

# Legend parameters
legend_x = 385  # X position for the legend
legend_y = 500  # Y position for the legend
rect_width = 20  # Width of color rectangles
rect_height = 20  # Height of color rectangles
spacing = 30  # Space between legend items
legend_padding = 10  # Padding around the legend inside the border

# Define legend values (5 values evenly spaced between min and max rainfall)
legend_values = [min_rainfall + i * (max_rainfall - min_rainfall) / 4 for i in range(5)]

# Calculate legend box height and width
legend_box_height = len(legend_values) * spacing + legend_padding * 2
legend_box_width = rect_width + 100  # Adjust this to fit text properly

# Add a rectangle as the legend border
legend_border = soup.new_tag('rect', x=str(legend_x - legend_padding), y=str(legend_y - legend_padding),
                             width=str(legend_box_width), height=str(legend_box_height),
                             stroke="black", fill="none", **{"stroke-width": "2"})
legend_group.append(legend_border)

# Loop to create each rectangle and label in the legend
for i, value in enumerate(legend_values):
    # Create a <rect> for the color
    rect = soup.new_tag('rect', x=str(legend_x), y=str(legend_y + i * spacing),
                        width=str(rect_width), height=str(rect_height),
                        fill=get_color(value))
    
    # Create a <text> for the label
    text = soup.new_tag('text', x=str(legend_x + rect_width + 10), y=str(legend_y + i * spacing + 15))
    text.string = f'{int(value)} mm'
    
    # Append the <rect> and <text> to the legend group
    legend_group.append(rect)
    legend_group.append(text)

# Append the legend group to the SVG
soup.svg.append(legend_group)

# Save the modified SVG with colored states, borders, and legend with border
with open('svg files/color_rainfall_india_with_legend_border.svg', 'w', encoding='utf-8') as output_file:
    output_file.write(str(soup))

print("SVG file updated with rainfall data, borders, and a legend with border.")


SVG file updated with rainfall data, borders, and a legend with border.
